# vari'art: 
### Example of latent analysis of a rap clip

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.layers import (
    InputLayer, 
    Dense, 
    Reshape, 
    Flatten, 
    Dropout, 
    Conv2D, 
    Conv2DTranspose, 
    MaxPool2D,
    BatchNormalization,
    LeakyReLU
)
from tensorflow.keras.optimizers import Adam

from variart.preprocessing import ArtVideo
from variart.model import VAE, GAN
from variart.latent import Latent

## 1. Load data and preprocessing

In [ ]:
# Load video
name = 'DrillFR4' 
filename = 'inputs/DrillFR4.mp4'
DrillFR4 = ArtVideo(name, filename)
DrillFR4.load_video()

# Crop images as squares
DrillFR4.square()

# Resize images
size = 128
new_shape=(size,size)
DrillFR4.resize(new_shape=new_shape)

# Rescale pixels in (0,1)
DrillFR4.rescale_images()

# Input data shape
print(f"Shape {DrillFR4.name}: {DrillFR4.shape}")

In [ ]:
# Show randomm image
DrillFR4.show_random_image()

## 2. Train GAN

In [ ]:
# Parameters
batch_size = 128
noise_dim = 256
learning_rate=1e-4
wgan=False # Wasserstein GAN configuration if True

In [ ]:
# Prepare data for training
data_train = DrillFR4.X.astype('float32')
data_train = shuffle(data_train, random_state=0)

input_shape_tuple = data_train.shape[1:]
train_dataset = tf.data.Dataset.from_tensor_slices(data_train).batch(batch_size)

In [ ]:
# Definition of functions to create the generator and the discriminator
def make_generator_model():
    dim=int(size/4)
    generative_net = tf.keras.Sequential(
        [
            Dense(units=dim*dim*32, use_bias=False, input_shape=(noise_dim,)),
            BatchNormalization(),
            LeakyReLU(),
            Reshape(target_shape=(dim, dim, 32)),
            Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', use_bias=False),
            BatchNormalization(),
            LeakyReLU(),
            Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', use_bias=False),
            BatchNormalization(),
            LeakyReLU(),
            Conv2DTranspose(filters=3, kernel_size=3, strides=1, padding='same', use_bias=False, activation='tanh'),
        ]
    )
    return generative_net

def make_discriminator_model(wgan=False):
    discriminative_net = tf.keras.Sequential([
        Conv2D(16, (5, 5), strides=(2, 2), padding='same', input_shape=[size, size, 3]),
        LeakyReLU(),
        Dropout(0.3),
        Conv2D(32, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(),
        Dropout(0.3),
        Flatten(),
    ])
    
    if wgan:
        discriminative_net.add(Dense(1))
    else:
        discriminative_net.add(Dense(1, activation='sigmoid'))
        
    return discriminative_net

In [ ]:
# Genertor and discriminator
generator = make_generator_model()
discriminator = make_discriminator_model(wgan=wgan)

In [ ]:
# Create GAN object
gan_model = GAN(DrillFR4.name, noise_dim, input_shape_tuple, generator, discriminator, learning_rate=learning_rate, wgan=wgan)

In [ ]:
# Train GAN
gan_model.train(train_dataset, epochs=1000, n_steps_gen=1, n_steps_disc=1, freq_plot=10, n_to_plot=4)

In [ ]:
# Generate images
gan_model.generate_and_plot(n_to_plot=4)

## 3. Train VAE

In [ ]:
# Prepare data for training
data = DrillFR4.X.astype('float32')
data = shuffle(data, random_state=0)

TRAIN_BUF = int(data.shape[0]*0.9)
data_train = data[:TRAIN_BUF]
data_validation = data[TRAIN_BUF:]

In [ ]:
# Parameters
batch_size = 128
epochs = 10000
early_stop_patience = 15
latent_dim = 16
optimizer = Adam(1e-3)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(data_train).batch(batch_size)
validation_dataset = tf.data.Dataset.from_tensor_slices(data_validation).batch(batch_size)
nb_features = data.shape[1]*data.shape[2]*data.shape[3]
input_shape = (batch_size, data.shape[1], data.shape[2], data.shape[3])

In [ ]:
# Encoder and decoder networks (inference and generative)
inference_net = tf.keras.Sequential(
      [
          InputLayer(input_shape=(data.shape[1], data.shape[2], data.shape[3])),
          Conv2D(filters=4, kernel_size=3, strides=(1, 1), activation='tanh'),
          MaxPool2D((2,2)),
          BatchNormalization(),
          Conv2D(filters=8, kernel_size=3, strides=(1, 1), activation='tanh'),
          MaxPool2D((2,2)),
          BatchNormalization(),
          Flatten(),
          Dense(latent_dim + latent_dim),
      ]
    )

generative_net = tf.keras.Sequential(
        [
            InputLayer(input_shape=(latent_dim,)),
            Dense(units=data.shape[1]*data.shape[2]*4, activation='tanh'),
            BatchNormalization(),
            Reshape(target_shape=(data.shape[1], data.shape[2], 4)),
            Conv2DTranspose(
              filters=8,
              kernel_size=3,
              strides=(1, 1),
              padding="SAME",
              activation='tanh'),
            BatchNormalization(),
            Conv2DTranspose(
              filters=4,
              kernel_size=3,
              strides=(1, 1),
              padding="SAME",
              activation='tanh'),
            BatchNormalization(),
            Conv2DTranspose(
              filters=3, kernel_size=3, strides=(1, 1), padding="SAME"),
        ]
    )

In [ ]:
# Model definition
model = VAE(DrillFR4.name, latent_dim, input_shape, inference_net, generative_net)

In [ ]:
# Train
model = model.train(optimizer, 
                    train_dataset, 
                    validation_dataset, 
                    epochs,
                    batch_size,
                    early_stop_patience = early_stop_patience, 
                    freq_plot = 25, 
                    plot_test = True,
                    n_to_plot = 4)

## 4. Latent analysis

In [ ]:
# Create latent object
LatentDrillFR4 = Latent(data, model)

In [ ]:
# Encode and decode data
LatentDrillFR4.encode_data()
LatentDrillFR4.decode_data()

In [ ]:
# Create tsne representation of data in latent space
LatentDrillFR4.latent_tsne()
LatentDrillFR4.plot_latent_tsne()

In [ ]:
# Compute distributions of latent space dimensions
LatentDrillFR4.compute_dist_coord()
LatentDrillFR4.plot_latent_dist_coord()

In [ ]:
# Perform clustering in latent space, test number of cluesters on a grid
LatentDrillFR4.latent_space_clustering(grid=range(5,100,5))
LatentDrillFR4.plot_silhouette_score()

In [ ]:
# Select number of clusters
n_clusters = 5
clusterer = LatentDrillFR4.dico_clust[n_clusters]['clusterer']
LatentDrillFR4.plot_latent_tsne(clusterer=clusterer)

In [ ]:
# Show images for a given cluster
label = 0
list_id = [i for i,l in enumerate(clusterer.labels_) if l==label][0:5]
LatentDrillFR4.plot_encoded_decoded(list_id=list_id)

In [ ]:
# Generate images by sampling from distributions in the latent space
list_z, fig = LatentDrillFR4.generate_image(n=5, method='dist')
fig.show()

In [ ]:
# Create a GIF from generated images
filename = f"outputs/gif_{LatentDrillFR4.name}.gif"
LatentDrillFR4.create_gif(list_z, filename)